In [2]:
import utils
import config
from finetune import finetune_model
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [3]:
import datasets_prep as dp
train_dataset = dp.get_toxigen_dataset("train")

Map: 100%|██████████| 940/940 [00:00<00:00, 9278.58 examples/s]


In [9]:
import pandas as pd
PATH_TO_DATA = "../../data/toxigen/"
val = pd.read_csv(PATH_TO_DATA + "val.csv")

In [13]:
val["target_group"].value_counts()

target_group
folks with physical disabilities    95
black/african-american folks        92
jewish folks                        87
muslim folks                        83
chinese folks                       77
mexican folks                       73
folks with mental disabilities      68
middle eastern folks                68
lgbtq+ folks                        66
women                               65
latino/hispanic folks               61
asian folks                         51
native american/indigenous folks    34
native american folks               20
Name: count, dtype: int64

In [ ]:
torch.load("../../output/IF_scores.pt")

In [ ]:
PATH_TO_DATA = "../../data/toxigen/"
annotated_train, annotated_test = utils.prepare_datasets()
tokenizer = AutoTokenizer.from_pretrained(config.TOKENIZER_NAME)

model_path = "../../output/base_model_finetuning/checkpoint-5600/"
model_checkpoint = AutoModelForSequenceClassification.from_pretrained(model_path,num_labels = 2).to("cuda")

train_dataloader = utils.get_dataloader(annotated_train, tokenizer, config.MAX_LENGTH, config.BATCH_SIZE)
val_dataloader = utils.get_dataloader(annotated_test, tokenizer, config.MAX_LENGTH, config.BATCH_SIZE)
first_module_baseline = utils.FirstModuleBaseline(annotated_train["text"].tolist(), annotated_test["text"].tolist(), model_checkpoint, tokenizer)

: 

In [3]:
#scores = first_module_baseline.get_Bm25_scores()
scores = torch.load("../../output/BM25_scores.pt")

In [4]:
scores.shape

torch.Size([940, 8960])

In [5]:
group_indices_train = annotated_train['target_group'].astype('category').cat.codes.tolist()
group_indices_val = annotated_test['target_group'].astype('category').cat.codes.tolist()

In [6]:
d3m_bm25 = utils.D3M(
        model=model_checkpoint,
        checkpoints=[],
        train_dataloader=train_dataloader,
        val_dataloader = val_dataloader,
        group_indices_train=group_indices_train,
        group_indices_val=group_indices_val,
        scores=scores,
        train_set_size=None,
        val_set_size=None,
        device="cuda")

In [7]:
debiased_train_indices = d3m_bm25.debias(num_to_discard=50)

In [ ]:
for name, module in model_checkpoint.named_modules():
    print(name)

In [ ]:
PATH_TO_DATA = "../../data/toxigen/"
annotated_train, annotated_test = utils.prepare_datasets()
tokenizer = AutoTokenizer.from_pretrained(config.TOKENIZER_NAME)

model_path = "../../output/base_model_finetuning/checkpoint-5600/"
model_checkpoint = AutoModelForSequenceClassification.from_pretrained(model_path,num_labels = 2).to("cuda")
train_dataset = utils.TextClassificationDataset(annotated_train["text"].tolist(), annotated_train["label"], tokenizer, config.MAX_LENGTH)
val_dataset = utils.TextClassificationDataset(annotated_test["text"].tolist(), annotated_test["label"], tokenizer, config.MAX_LENGTH)
train_dataloader = utils.get_dataloader(annotated_train, tokenizer, config.MAX_LENGTH, config.BATCH_SIZE)
val_dataloader = utils.get_dataloader(annotated_test, tokenizer, config.MAX_LENGTH, config.BATCH_SIZE)

: 

In [3]:
first_module_tda = utils.FirstModuleTDA(train_dataloader, val_dataloader,train_dataset, model_checkpoint,tokenizer)
first_module_tda.get_IF_scores(out = "../output/IF_scores")

Fitting covariance matrices [280/280] 100%|██████████ [time left: 00:00, time spent: 04:26]
Performing Eigendecomposition [74/74] 100%|██████████ [time left: 00:00, time spent: 00:43]
Fitting Lambda matrices [280/280] 100%|██████████ [time left: 00:00, time spent: 24:45]


TypeError: 'DataLoader' object is not subscriptable

In [4]:
first_module_tda = utils.FirstModuleTDA(train_dataloader, val_dataloader, model_checkpoint,tokenizer)
first_module_tda.get_TRAK_scores(out = "../../output/TRAK_scores/")

ERROR:TRAK:Could not use CudaProjector.
Reason: No module named 'fast_jl'
ERROR:TRAK:Defaulting to BasicProjector.
/home/oh/owl982/.conda/envs/tda_for_fairness/lib/python3.12/site-packages/trak/traker.py:69: SyntaxWarning: invalid escape sequence '\e'
  """


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.39 GiB. GPU 